<a href="https://colab.research.google.com/github/un1u3/ml-labs/blob/main/Deep%20Learning/Pytorch/04_Example_Binary_Classification_with_the_Wine_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

In [ ]:
import torch
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# load the wine dataser
data = load_wine()
X = data.data
y = data.target

# convert the task to binary classification
y = (y == 0).astype(float)

# split into train and test sets
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2,random_state=42)

# scale the features
scaler = StandardScaler()
X_train_scale  = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

# convert to Pytorch tensor
X_train = torch.tensor(X_train,dtype = torch.float)
y_train = torch.tensor(y_train,dtype= float).unsqueeze(1)
X_test = torch.tensor(X_test,dtype=float)
y_test = torch.tensor(y_test,dtype=float).unsqueeze(1)


# createdata loders
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test,y_test)

train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)


# define a Neural Network

In [ ]:
import torch.nn as nn

class WineClassifier(nn.Module):
  def __init__(self):
    super(WineClassifier,self).__init__()
    self.network = nn.Sequential(
        nn.Linear(13,32), # Input layer : 13 features to 32 neurons
        nn.ReLU(), # activation function
        nn.Linear(32,16), # hidden  layer
        nn.ReLU(), # activation function
        nn.Linear(16,1), # Output layer: 1 neuron for binary classification
        nn.Sigmoid() # Sigmoid activation for binary output
        )

  def forward(self,x):
    return self.network(x)

In [ ]:
model = WineClassifier()

In [ ]:
model

WineClassifier(
  (network): Sequential(
    (0): Linear(in_features=13, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Sigmoid()
  )
)

# define loss function and optimizer

In [ ]:
import torch.optim as optim

# loss function
criterion = nn.BCELoss()

# optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)

# Training the model

In [ ]:
# Training loop
epochs = 69
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for batch in train_loader:
        X_batch, y_batch = batch

        # Forward pass
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)

        # Backward pass
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update parameters

        running_loss += loss.item()

    # Calculate average training loss
    avg_train_loss = running_loss / len(train_loader)

    # Validation step
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for batch in test_loader: # Changed val_loader to test_loader
            X_batch, y_batch = batch
            predictions = model(X_batch)
            val_loss += criterion(predictions, y_batch).item()

            # Calculate accuracy
            predicted_labels = (predictions > 0.5).float()
            correct += (predicted_labels == y_batch).sum().item()
            total += y_batch.size(0)

    avg_val_loss = val_loss / len(test_loader) # Changed val_loader to test_loader
    accuracy = correct / total

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/69
  Training Loss: 31.4749
  Validation Loss: 53.8664, Accuracy: 0.6111
Epoch 2/69
  Training Loss: 28.7650
  Validation Loss: 53.8666, Accuracy: 0.6111
Epoch 3/69
  Training Loss: 26.5372
  Validation Loss: 53.8667, Accuracy: 0.6111
Epoch 4/69
  Training Loss: 27.1815
  Validation Loss: 53.8668, Accuracy: 0.6111
Epoch 5/69
  Training Loss: 29.1552
  Validation Loss: 53.8670, Accuracy: 0.6111
Epoch 6/69
  Training Loss: 28.7278
  Validation Loss: 53.8672, Accuracy: 0.6111
Epoch 7/69
  Training Loss: 27.0927
  Validation Loss: 53.8673, Accuracy: 0.6111
Epoch 8/69
  Training Loss: 29.2368
  Validation Loss: 53.8675, Accuracy: 0.6111
Epoch 9/69
  Training Loss: 28.7296
  Validation Loss: 53.8676, Accuracy: 0.6111
Epoch 10/69
  Training Loss: 27.3445
  Validation Loss: 53.8678, Accuracy: 0.6111
Epoch 11/69
  Training Loss: 28.3686
  Validation Loss: 53.8679, Accuracy: 0.6111
Epoch 12/69
  Training Loss: 28.5079
  Validation Loss: 53.8680, Accuracy: 0.6111
Epoch 13/69
  Training Lo

In [ ]:
# code